In [64]:
%load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths

# import mplfinance as mpf
%matplotlib inline

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300, 'lines.markersize': 4})
plt.rcParams['lines.linewidth'] = 0.5 



The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
exchange = 'kucoin'
pair = 'BTC_USDT'
timeframe = '4h'

odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()

odf

,timestamp,open,high,low,close,volume,date,idate
date,,,,,,,,
2020-01-01 00:00:00,1577836800000,7196.2,7243.2,7176.1,7225.4,62.743859,2020-01-01 00:00:00,20200101
2020-01-01 04:00:00,1577851200000,7225.4,7233.8,7203.1,7209.5,36.248625,2020-01-01 04:00:00,20200101
2020-01-01 08:00:00,1577865600000,7209.4,7232.0,7183.0,7195.6,52.545595,2020-01-01 08:00:00,20200101
2020-01-01 12:00:00,1577880000000,7195.7,7251.8,7195.6,7231.4,69.704852,2020-01-01 12:00:00,20200101
2020-01-01 16:00:00,1577894400000,7231.5,7248.0,7217.1,7229.0,60.889127,2020-01-01 16:00:00,20200101
...,...,...,...,...,...,...,...,...
2023-07-06 16:00:00,1688659200000,30406.6,30464.7,30159.5,30311.5,647.921674,2023-07-06 16:00:00,20230706
2023-07-06 20:00:00,1688673600000,30311.4,30342.6,29855.4,29895.0,683.000406,2023-07-06 20:00:00,20230706
2023-07-07 00:00:00,1688688000000,29895.0,30164.0,29701.0,30139.8,715.228676,2023-07-07 00:00:00,20230707


In [6]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .1, .02
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')

In [90]:

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))

In [91]:

### Ranges
# nest_asyncio.apply()

strategy = 'swings'
strategy_params_json = f'./par-{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})

dlen = len(odf)

sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
wlen = int(pow(2,sl_n2.value)-1)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/wlen), step=1, value=1)
def update_sl_w_range(*args):
    wlen = int(pow(2,sl_n2.value) - 1)
    sl_w.max = ceil(dlen/wlen)
    wdg_inow.max = wlen
    wdg_inow.value = wlen
sl_n2.observe(update_sl_w_range, 'value')

sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
sl_lag_n = IntSlider(value=5,min=1,max=50,step=1,description='lag_n',continuous_update=False)
# tx_max_rat = FloatText(value=0.05,min=0.01,max=0.1,step=0.001,description='max_rat',continuous_update=False)
sl_rat = FloatRangeSlider(value=(0.02,0.05),min=0.01,max=0.2,step=0.001,description='sl_rat',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)
wdg_pkdist = IntSlider(value=7,min=1,max=100,step=1,description='PKDist',continuous_update=False)
wdg_inow = IntSlider(value=0,min=0,max=wlen,step=1,description='inow',continuous_update=False)
wdg_pkprom = FloatRangeSlider(value=(0.05,1),min=0,max=1,step=0.005,description='PKProm',continuous_update=False)

strategy_params = {
    "inow": {'wdg': wdg_inow},
    "hold": {'wdg': sl_hold},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir},
    "lag_n": {"wdg": sl_lag_n},
    "pkdist": {"wdg": wdg_pkdist},
    "pkprom": {"wdg": wdg_pkprom},
    "rat": {"wdg": sl_rat}
}
all_params = {
    'w2log': {'wdg': sl_n2}, 'w': {'wdg': sl_w},
    **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in all_params.items()]

ui = widgets.VBox([
    # widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in all_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf=None
def printer(
        w2log, w, inow, hold, candles, mdir, lag_n, rat, pkdist, pkprom
):
    global wdf

    glag_n = lag_n
    min_rat, max_rat = rat
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:]#.copy()    
    wlen = len(wdf)

    print(f'N={len(wdf)}; Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')
    # print(f'lag_n={lag_n}; min_rat={min_rat}; max_rat={max_rat}\n')

    peaks, peak_props = find_peaks(wdf.close[:inow], prominence=100, distance=pkdist)
    results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    peaks_idx = wdf.index[peaks]
    close_peaks = wdf.loc[peaks_idx].close
    fig, (ax1,ax2, ax3) = plt.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

    ax1.axvline(wdf.index[inow])

    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
    ax1.plot(peaks_idx, close_peaks, "^", c='m', markersize=2, alpha=0.5)
    ax1.vlines(x=peaks_idx, ymin=close_peaks - peak_props["prominences"],
           ymax = close_peaks, color = "C1", lw=0.2)
    ax1.hlines(results_full[1:][0], xmin=wdf.index[results_full[1:][1].astype(int)], xmax=wdf.index[results_full[1:][2].astype(int)], color="C1", lw=0.3)
    ax1.hlines(results_half[1:][0], xmin=wdf.index[results_half[1:][1].astype(int)], xmax=wdf.index[results_half[1:][2].astype(int)], color="C2", lw=0.3)
    # ax1.hlines(y=(close_peaks - peak_props["prominences"]), xmin=wdf.index[peak_props["left_bases"]],
    #        xmax=wdf.index[peak_props["right_bases"]], color = "C1")


    # cid = fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()
    
out = widgets.interactive_output(printer, {
        **{k : v['wdg'] for k,v in all_params.items()}
    });
x = display(ui, out);



Output()

In [76]:

peaks, peak_props = find_peaks(wdf.close, prominence=1, distance=10)
# wdf.iloc[peaks].close

results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
# np.shape(results_half[1:])
results_half[1:][1].astype(int)
# fig, (ax1,ax2, ax3) = plt.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

# plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
# plt.plot(peaks, wdf.iloc[peaks].close, "x")

array([  1,   9,  24,  35,  49,  61,  81,  91, 101, 112, 127, 142, 159,
       172, 189, 202, 218, 229, 245, 240, 266, 279, 302, 316, 320, 338,
       347, 358, 366, 383, 381, 406, 418, 428, 439, 450, 463, 480, 501])

In [56]:
tuple([1,2])

(1, 2)